In [1]:
#setting up the document
import eikon as ek
import datetime
from datetime import datetime
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy
import seaborn as sns
import xlsxwriter
import math
from math import exp, sqrt, log
%matplotlib inline
from scipy import stats
import statsmodels as sm
from statsmodels.regression.linear_model import OLS
import sqlite3
from sqlite3 import Error

In [2]:
ek.set_app_key('dfd3e4f6ef7143e8bf90ac68e33c1499304694ed')

In [3]:
#creating list containing RICs of analysed companies
rics = ["JPM.N","BAC.N","C.N","WFC.N", "GS","MS.N","HSBA.L","BNPP.PA","CAGR.PA","SAN.MC","SOGN.PA","BARC.L"]
 
#retrieving the Close Price and the Date for the last 10 years for all RICs
price_close = ek.get_data(rics,['TR.ClosePrice.Date','TR.ClosePrice'], {'Sdate':'2002-01-01', 'EDate':'2021-12-31'})
price_close

(      Instrument                  Date  Close Price
 0          JPM.N  2002-01-01T00:00:00Z         <NA>
 1          JPM.N  2002-01-02T00:00:00Z        36.35
 2          JPM.N  2002-01-03T00:00:00Z        37.32
 3          JPM.N  2002-01-04T00:00:00Z         39.0
 4          JPM.N  2002-01-07T00:00:00Z         38.9
 ...          ...                   ...          ...
 60794     BARC.L  2021-12-23T00:00:00Z        187.4
 60795     BARC.L  2021-12-24T00:00:00Z       187.44
 60796     BARC.L  2021-12-29T00:00:00Z        188.5
 60797     BARC.L  2021-12-30T00:00:00Z       188.06
 60798     BARC.L  2021-12-31T00:00:00Z        187.0
 
 [60799 rows x 3 columns],
 None)

In [4]:
#grouping the results by RICs and concating all df
dfs = dict(tuple(price_close[0].groupby('Instrument'))) 
dfarray = [] 
for ric, data in dfs.items(): 
   df_tmp = dfs[ric].dropna() 
   df_tmp = df_tmp.drop_duplicates() 
   df_tmp = df_tmp.set_index('Date')
   df_tmp = df_tmp.drop(['Instrument'], axis=1) 
   df_tmp = df_tmp.rename(columns={"Close Price":ric}) 
   dfarray.append(df_tmp) 
     
price_close_converted= pd.concat(dfarray, axis=1, sort=True) 
price_close_converted.columns.name = 'CLOSE'

#creating a dataframe, changing the inxed to Datetime and droping the Timestamp
df_price_close = pd.DataFrame(price_close_converted)
df_price_close.index = pd.to_datetime(df_price_close.index).strftime('%Y-%m-%d')
df_price_close

CLOSE,BAC.N,BARC.L,BNPP.PA,C.N,CAGR.PA,GS,HSBA.L,JPM.N,MS.N,SAN.MC,SOGN.PA,WFC.N
Date,,,,,,,,,,,,
2002-01-02,31.48,509.406461,48.362064,510.1,15.947297,91.07,706.734596,36.35,46.168062,5.593177,55.256933,21.72
2002-01-03,31.445,514.579242,48.844238,510.5,15.974326,92.45,724.163316,37.32,47.505782,5.676303,56.402611,21.8
2002-01-04,31.935,517.727891,49.229977,520.0,15.938287,96.75,731.134804,39.0,49.863513,5.711928,58.209257,21.855
2002-01-07,31.555,511.430593,49.085325,514.3,16.379766,95.75,724.163316,38.9,49.487279,5.462551,56.622934,21.89
2002-01-08,31.035,509.181558,49.18176,495.0,16.668079,92.5,716.320392,38.6,48.433825,5.545676,57.107643,21.805
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,44.64,<NA>,60.38,60.65,12.504,388.04,<NA>,158.16,100.4,2.9235,29.99,48.77
2021-12-28,44.7,<NA>,60.82,60.62,12.55,387.61,<NA>,158.64,99.97,2.931,30.145,48.42
2021-12-29,44.63,188.5,60.71,60.54,12.568,386.2,449.4,158.56,98.73,2.9225,30.275,48.45


In [5]:
#checking how many NAs are in each column to get a feeling on how many rows will be deleted
df_price_close.isna().sum(axis = 0)

CLOSE
BAC.N      131
BARC.L     111
BNPP.PA     45
C.N        131
CAGR.PA     45
GS         131
HSBA.L     111
JPM.N      131
MS.N       131
SAN.MC      75
SOGN.PA     45
WFC.N      131
dtype: int64

In [6]:
#droping rows with NAs in it
df_price_close_corr = df_price_close.dropna()
df_price_close_corr

CLOSE,BAC.N,BARC.L,BNPP.PA,C.N,CAGR.PA,GS,HSBA.L,JPM.N,MS.N,SAN.MC,SOGN.PA,WFC.N
Date,,,,,,,,,,,,
2002-01-02,31.48,509.406461,48.362064,510.1,15.947297,91.07,706.734596,36.35,46.168062,5.593177,55.256933,21.72
2002-01-03,31.445,514.579242,48.844238,510.5,15.974326,92.45,724.163316,37.32,47.505782,5.676303,56.402611,21.8
2002-01-04,31.935,517.727891,49.229977,520.0,15.938287,96.75,731.134804,39.0,49.863513,5.711928,58.209257,21.855
2002-01-07,31.555,511.430593,49.085325,514.3,16.379766,95.75,724.163316,38.9,49.487279,5.462551,56.622934,21.89
2002-01-08,31.035,509.181558,49.18176,495.0,16.668079,92.5,716.320392,38.6,48.433825,5.545676,57.107643,21.805
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-21,44.19,184.26,58.86,59.38,12.174,380.32,445.2,156.09,97.77,2.811,29.12,47.81
2021-12-22,44.28,185.84,59.35,59.6,12.252,382.33,444.6,156.7,98.3,2.8155,29.44,48.03
2021-12-23,44.42,187.4,60.0,60.21,12.44,385.04,447.9,157.26,99.34,2.9015,29.83,48.36


In [7]:
#double-checking
df_price_close_corr.isna().sum(axis = 0)

CLOSE
BAC.N      0
BARC.L     0
BNPP.PA    0
C.N        0
CAGR.PA    0
GS         0
HSBA.L     0
JPM.N      0
MS.N       0
SAN.MC     0
SOGN.PA    0
WFC.N      0
dtype: int64

In [8]:
# adding date column to dataframe
index1=df_price_close_corr.index
df_price_close_corr['Date1']=index1
df_price_close_corr.head(5)

<ipython-input-8-7680461c5af4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_price_close_corr['Date1']=index1


CLOSE,BAC.N,BARC.L,BNPP.PA,C.N,CAGR.PA,GS,HSBA.L,JPM.N,MS.N,SAN.MC,SOGN.PA,WFC.N,Date1
Date,,,,,,,,,,,,,
2002-01-02,31.48,509.406461,48.362064,510.1,15.947297,91.07,706.734596,36.35,46.168062,5.593177,55.256933,21.72,2002-01-02
2002-01-03,31.445,514.579242,48.844238,510.5,15.974326,92.45,724.163316,37.32,47.505782,5.676303,56.402611,21.8,2002-01-03
2002-01-04,31.935,517.727891,49.229977,520.0,15.938287,96.75,731.134804,39.0,49.863513,5.711928,58.209257,21.855,2002-01-04
2002-01-07,31.555,511.430593,49.085325,514.3,16.379766,95.75,724.163316,38.9,49.487279,5.462551,56.622934,21.89,2002-01-07
2002-01-08,31.035,509.181558,49.18176,495.0,16.668079,92.5,716.320392,38.6,48.433825,5.545676,57.107643,21.805,2002-01-08


In [9]:
#getting the info on the dataframe again
df_price_close_corr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4907 entries, 2002-01-02 to 2021-12-30
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   BAC.N    4907 non-null   Float64
 1   BARC.L   4907 non-null   Float64
 2   BNPP.PA  4907 non-null   Float64
 3   C.N      4907 non-null   Float64
 4   CAGR.PA  4907 non-null   Float64
 5   GS       4907 non-null   Float64
 6   HSBA.L   4907 non-null   Float64
 7   JPM.N    4907 non-null   Float64
 8   MS.N     4907 non-null   Float64
 9   SAN.MC   4907 non-null   Float64
 10  SOGN.PA  4907 non-null   Float64
 11  WFC.N    4907 non-null   Float64
 12  Date1    4907 non-null   object 
dtypes: Float64(12), object(1)
memory usage: 594.2+ KB


In [10]:
# convert data from dataframe into a vector of lists
records_to_insert = df_price_close_corr.values.tolist()
records_to_insert

[[31.48,
  509.406461,
  48.362064,
  510.1,
  15.947297,
  91.07,
  706.734596,
  36.35,
  46.168062,
  5.593177,
  55.256933,
  21.72,
  '2002-01-02'],
 [31.445,
  514.579242,
  48.844238,
  510.5,
  15.974326,
  92.45,
  724.163316,
  37.32,
  47.505782,
  5.676303,
  56.402611,
  21.8,
  '2002-01-03'],
 [31.935,
  517.727891,
  49.229977,
  520.0,
  15.938287,
  96.75,
  731.134804,
  39.0,
  49.863513,
  5.711928,
  58.209257,
  21.855,
  '2002-01-04'],
 [31.555,
  511.430593,
  49.085325,
  514.3,
  16.379766,
  95.75,
  724.163316,
  38.9,
  49.487279,
  5.462551,
  56.622934,
  21.89,
  '2002-01-07'],
 [31.035,
  509.181558,
  49.18176,
  495.0,
  16.668079,
  92.5,
  716.320392,
  38.6,
  48.433825,
  5.545676,
  57.107643,
  21.805,
  '2002-01-08'],
 [30.975,
  515.029049,
  50.242543,
  497.9,
  16.668079,
  93.75,
  700.198826,
  38.71,
  48.818419,
  5.492238,
  57.416095,
  21.695,
  '2002-01-09'],
 [31.0,
  517.278084,
  50.001456,
  498.5,
  16.397785,
  93.0,
  691.048

In [11]:
#creating a database connection tool to a SQLite database

def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()
 
 
if __name__ == '__main__':
    create_connection(r"/Users/lucarizzo/Dropbox/Ausbildung/Studium/ZHAW/Master/Semester_2/Advanced_Quantitative_Methods/Semesterabeit/EventStudy1.db")

2.6.0


In [12]:
conn = sqlite3.connect(r"/Users/lucarizzo/Dropbox/Ausbildung/Studium/ZHAW/Master/Semester_2/Advanced_Quantitative_Methods/Semesterabeit/EventStudy1.db")

print("Opened database successfully")

Opened database successfully


In [13]:
# Creating a table in the SQL database "EventStudy1"
conn = sqlite3.connect('EventStudy1.db')
c = conn.cursor()
c.execute('''CREATE TABLE Close_Price_corrected
         (DATE TIMESTAMP,
         BAC_N          INT    ,
         BARC_L         INT    ,
         BNPP_PA        INT    ,
         C_N            INT    ,
         CAGR_PA        INT    ,
         GS             INT    ,
         HSBA_L         INT    ,
         JPM_N          INT    ,
         MS_N           INT    ,
         SAN_MC         INT    ,
         SOGN_PA        INT    ,
         WFC_N INT)''')

print("Table created successfully")

Table created successfully


In [14]:
#uploading the data from the vector list "records_to_insert" to our database

c = conn.cursor()
c.executemany("INSERT INTO Close_Price_corrected(BAC_N,BARC_L,BNPP_PA,C_N,CAGR_PA,GS,HSBA_L,JPM_N,MS_N,SAN_MC,SOGN_PA,WFC_N,DATE) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)", records_to_insert)    
conn.commit()

In [15]:
#testing the upload and pulling the data from the database to python 
c = conn.cursor()
c.execute("SELECT * FROM Close_Price_corrected")
new_data=c.fetchall()
new_data

[('2002-01-02',
  31.48,
  509.406461,
  48.362064,
  510.1,
  15.947297,
  91.07,
  706.734596,
  36.35,
  46.168062,
  5.593177,
  55.256933,
  21.72),
 ('2002-01-03',
  31.445,
  514.579242,
  48.844238,
  510.5,
  15.974326,
  92.45,
  724.163316,
  37.32,
  47.505782,
  5.676303,
  56.402611,
  21.8),
 ('2002-01-04',
  31.935,
  517.727891,
  49.229977,
  520,
  15.938287,
  96.75,
  731.134804,
  39,
  49.863513,
  5.711928,
  58.209257,
  21.855),
 ('2002-01-07',
  31.555,
  511.430593,
  49.085325,
  514.3,
  16.379766,
  95.75,
  724.163316,
  38.9,
  49.487279,
  5.462551,
  56.622934,
  21.89),
 ('2002-01-08',
  31.035,
  509.181558,
  49.18176,
  495,
  16.668079,
  92.5,
  716.320392,
  38.6,
  48.433825,
  5.545676,
  57.107643,
  21.805),
 ('2002-01-09',
  30.975,
  515.029049,
  50.242543,
  497.9,
  16.668079,
  93.75,
  700.198826,
  38.71,
  48.818419,
  5.492238,
  57.416095,
  21.695),
 ('2002-01-10',
  31,
  517.278084,
  50.001456,
  498.5,
  16.397785,
  93,
  6